## General setup for a model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv('&todo')
#&cell
# take a look at the data
df.head()
#&cell

# Going to get a list of the categorical variables an the numeric valiables and take a look at a baseline
# model
target = '' #&todo

cat=df.select_dtypes(exclude='number')
num=df.select_dtypes(exclude='object')

# filter out unwanted features like dates
ignore=[''] #&todo
cat=[x for x in cat if x not in ignore]
num=[x for x in num if x not in ignore]
features=cat+num

# validation split
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,random_state=42)
train,val=train_test_split(train,random_state=42)

# feature sets
X_train=train[features]
X_val=val[features]
X_test=test[features]

y_train=train[target]
y_val=val[target]
y_test=test[target]

print("(train shape)(val shape)(test shape)")
train.shape,val.shape,test.shape
#&cell
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from category_encoders import OneHotEncoder
from sklearn.metrics import accuracy_score,r2_score

from sklearn.linear_model import [LinearRegression,LogisticRegression]

pipeline=make_pipeline(
#&todo make a model slection and encoding selection
)

pipeline.fit(X_train,y_train)
print("training r^2 is:",pipeline.score(X_train,y_train))
print("validation r^2 is:",pipeline.score(X_val,y_val))

#&cell


## pdp plot single

In [ ]:
from pdpbox.pdp import pdp_isolate,pdp_plot
# select the feature that is going to be looked at
feature = ''#&todo
# use a fitted model,df,a list of columns from the df, and the feature that you want to inspect to ini the pdp
isolate=pdp_isolate(
    model=model,
    dataset=X_val,
    model_features=X_val.columns,
    feature=feature
)
# use the pdp plot method with the pdp object and the target feature name to plot a graph
pdp_plot(isolate,feature_name=feature) #plot_lines=True,frac_to_plot=0.01 makes it so that it will track indeividuels instead of a proba cone

## pdp plot multi inspect

In [ ]:
from pdpbox.pdp import pdp_interact,pdp_interact_plot
# select features to use for the pdp plot
features=['Annual Income','Credit Score']
# diffrent object same process
interaction=pdp_interact(
        model=model,
        dataset=X_val,
        model_features=X_val.columns,
        features=features

)
# make a bit of a difffrent plot because of some bugs in this version of pdp
# use plot_type='grid'
pdp_interact_plot(interaction,plot_type='grid',feature_names=features)

## shaply plot


In [ ]:
# STUDY/PRACTICE THIS CELL FOR THE SPRINT CHALLENGE
import shap

processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

# To explain the prediction for test observation with index #3094, 
# first, get all of the features for that observation
row = X_test.iloc[[3094]]
row

explainer = shap.TreeExplainer(model)
row_processed = processor.transform(row)
shap_values = explainer.shap_values(row_processed)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values, 
    features=row, 
    link='logit' # For classification, this shows predicted probabilities
                 # otherwise it can be left omitted for regression problems
)


## getting the permutation importance of features

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

# make a transformation pipeline
autobot=make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median') # change this as needed
)

# make the transformed feature matrixs
X_train_encoded=autobot.fit_transform(X_train,y_train)
X_val_encoded=autobot.transform(X_val)

# fit the model with the transformed information
model=LinearRegression(n_jobs=-1)
model.fit(X_train_encoded,y_train)

# make a permutation importance finder
permuter=PermutationImportance(
    model,
    scoring=None,
    n_iter=5,
    random_state=42
)
# fit the premuter to the validation set
permuter.fit(X_val_encoded,y_val)

feature_names = X_val.columns.tolist()
# if this breaks look at the code from 2-3 LS_DS_234_assignment
pd.Series(permuter.feature_importances_, feature_names).sort_values().index

# actually shows the graph for the permution importance
eli5.show_weights(
    permuter,
    top=None,
    feature_names=feature_names
)